<a href="https://colab.research.google.com/github/tsakailab/alpp/blob/main/colab/alpp_model_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像認識のCNNモデルの構造を覗いてみよう

画像認識のニューラルネットワークは，局所的な特徴（断片的な形状，模様，配色）の組合せを調べるため，畳み込み層，活性化層，プーリング層，全結合層などの部品から構成されています．

## 学習済みのCNNモデルをTorchvisionから入手します．

[If pretrained=True, returns a model pre-trained on ImageNet.](https://pytorch.org/vision/stable/models.html)

In [ ]:
import torch
from torchvision import models

model = models.alexnet(pretrained=True)
#model = models.vgg16(pretrained=True)
#model = models.vgg16_bn(pretrained=True)
#model = models.resnet50(pretrained=True)
#model = models.googlenet(pretrained=True)
#model = models.mobilenet_v3_small(pretrained=True)
#model = models.efficientnet_b0(pretrained=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

### torch.nn.Moduleで構成されたモデルはprintで表示できます．

In [ ]:
print(model)

### 各層の**特徴マップ（feature maps）**のサイズと**学習可能なパラメータ（trainable parameters）**の総数を**torchsummary**で表示します．

- image_sizeは，想定する入力画像のサイズです（例：vgg16やalexnetでは32x32以上）．モデルによって，AdaptiveAvgPool2dまたはAdaptiveMaxpool2dで強制的に所定の大きさになります（vgg16では[-1,512,7,7]，alexnetでは[-1,256,6,6]）．
- 画像認識の特徴マップは4Dテンソルです．Output Shape は [B, C, H, W]．B:バッチサイズ，C:チャンネル数（特徴マップの枚数），H:高さ，W:幅．
- 特にバッチサイズを指定しなければ，B=-1として表示されます．
    - 入力データの数（入力画像の枚数）をバッチサイズといいます．つまり，画像を複数枚まとめてmoodelに入力できる仕様です．1枚ずつ入力しなくても，各入力画像に対する認識結果の出力をまとめて得られます．
    - 例：1000カテゴリのImageNetを学習したmodelを使い，サイズ224x224のカラー画像16枚の各認識結果をまとめて得るとき，入力のShapeは [16, 3, 224, 224]，出力のShapeは [16, 1000] です．

In [ ]:
from torchsummary import summary
image_size=224
summary(model, (3, image_size, image_size), device=device)

### [**torchinfo**](https://github.com/TylerYep/torchinfo)のsummaryで表示します．

こちらの方が新しく，詳しいです．カーネルサイズや積和の計算量も表示できます．

In [ ]:
!pip install torchinfo
from torchinfo import summary

batch_size, image_size = 1, 224
summary(model, (batch_size, 3, image_size, image_size))

In [ ]:
batch_size, image_size = 1, 224
summary(model, (batch_size, 3, image_size, image_size), 
        col_names=["input_size",
                    "output_size",
                    "num_params",
                    "kernel_size",
                    "mult_adds",
        ],
        row_settings=["var_names"])

In [ ]:
batch_size, image_size = 1, 224
summary(model, (batch_size, 3, image_size, image_size), depth=0)

## 学習済みのモデルを[PyTorch Hub](https://pytorch.org/hub/research-models)からも入手できます．

In [ ]:
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
#model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16_bn', pretrained=True) #vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19, vgg19_bn
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True) # resnet18, resnet34, resnet50, resnet101, resnet152
#model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
#model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
#model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50')
#model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet', type='efficientnet-widese-b0')
#model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
from torchsummary import summary
input_size=300
summary(model, (3, input_size, input_size), device=device)

# 要点
<!--
畳み込み演算によって，カーネルに類似した模様が見つかる場所と量（局所特徴量）を表す出力画像が得られることを理解できたでしょうか．このように局所特徴量の分布を表す画像を**特徴マップ（feature map）**といいます．

カーネルを何種類も用意すれば，エッジや凹凸の断片，色の組合せなど，様々な局所特徴について特徴マップを作成できます．３枚の入力で表せるカラー画像から特徴マップを出力するときと同様に，複数の特徴マップからひとつの特徴マップを作ることもできます．

様々な局所特徴量の分布を表す複数の特徴マップに対する畳み込み演算は，局所特徴の典型的な組合せを探すことに相当します．出力の特徴マップは，典型的な局所特徴の組合せが見つかる場所と量を表しています．
-->

畳み込み演算をする**畳み込み層（convolutional layer）**は，複数の特徴マップから複数の特徴マップを作成します．入力（出力）の特徴マップは，入力（出力）チャンネル（input/output channels）とも呼ばれます．

畳み込み層を何段ももつ**畳み込みニューラルネットワーク（convolutional neural network; CNN）**は，典型的な局所特徴の組合せ，その典型的な組合せの組合せ，さらにその組合せ・・を探す機構です．畳み込み層の間には適宜，**活性化層（activation layer）**や**プーリング層（pooling layer）**などが入ります．

活性化層は，特徴量を強調または選び出す処理により，典型的な局所特徴を見つけやすくします．プーリング層は特徴マップをぼかしてサイズを縮小します．縮小すると，局所特徴を広目に探せるようになるからです．

こららの層で構成された**特徴抽出器（feature extractor）**によって，入力画像がもつ特徴を定量化できます．**全結合層（fully connected (fc) layers）**は，特徴抽出器の出力から目的の量を算出します．物体認識の場合，全結合層は，想定している各物体が存在する確率を算出する**識別器（classifier）**の役割を担います．

<!--
![](https://www.researchgate.net/publication/329790469/figure/fig2/AS:705721712787456@1545268576139/Simplified-illustration-of-the-AlexNet-architecture.ppm)
![](https://upload.wikimedia.org/wikipedia/commons/6/63/Typical_cnn.png)
-->

# 参考資料
[VGG](https://arxiv.org/abs/1409.1556)，
[ResNet](https://arxiv.org/abs/1512.03385)，
[Inception](https://arxiv.org/abs/1512.00567)，
[GoogLeNet](https://arxiv.org/abs/1409.4842)，
[参考1](https://medium.com/zylapp/review-of-deep-learning-algorithms-for-image-classification-5fdbca4a05e2)，
[参考2](https://www.topbots.com/important-cnn-architectures/)

![](https://www.researchgate.net/profile/Evgeny-Baraboshkin/publication/335989937/figure/fig3/AS:806501341999104@1569296312136/Neural-networks-architecture-simplified-sketches-inception-and-residual-modules.png)
Neural networks architecture simplified sketches, inception and residual modules structures can be observed in the detailed network maps in online repository ("Supplementary materials to the article," n.d.). 0 -convolutoion layer, 1 -activation layer, 2 -Inception module, 3 -average pooling, 4 -batch normalization, 5 -max pooling, 6 -zero padding, 7 -Residual module, 8 -drop out layer, 9 -layer composition, 10 -dense layer.


![](https://drek4537l1klr.cloudfront.net/elgendy/v-3/Figures/05_27.png)
![](https://i.ytimg.com/vi/VxhSouuSZDY/maxresdefault.jpg)

